In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./datasets/final/24k_final.csv')
df

,label,URL_Length,Url_Shortening,Ssl_Info,entropy_subdomain,entropy_domain,Global_Ranking,Has_Suspicious_Params,Num_Languages,Uses_HTTPS,...,percent_Dots_Query,percent_Dash_Query,percent_AtSymbol_Query,percent_TildeSymbol_Query,percent_Underscore_Query,percent_Percent_Query,percent_Ampersand_Query,percent_Hash_Query,percent_Hex_Encoded_Characters_Query,Has_Fragment
0,0,11,False,NaN,0.000000,0.000000,NaN,False,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False
1,1,62,False,NaN,1.000000,3.926734,NaN,False,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False
2,1,61,False,NaN,1.918296,1.921928,NaN,False,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False
3,1,64,False,NaN,1.000000,3.819114,NaN,False,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False
4,1,162,False,NaN,2.521641,-0.000000,NaN,False,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.0,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,12,False,NaN,0.000000,0.000000,NaN,False,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False
23996,1,39,False,NaN,2.251629,3.826875,NaN,False,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False
23997,0,19,False,NaN,0.000000,0.000000,NaN,False,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False
23998,0,21,False,NaN,0.000000,0.000000,NaN,False,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,False


In [3]:
oldColumns = df.columns
df.sample(frac=1).reset_index(drop=True)

if 'Ssl_Info' in df.columns: 
    df.drop(['Ssl_Info'], axis=1, inplace=True)
    print('Ssl_Info dropped')
if 'Unnamed: 0' in df.columns:
    df.drop(['Unnamed: 0'], axis=1, inplace=True)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

nonNumeric = df.select_dtypes(include=['object']).columns
boolCols = df.select_dtypes(include=['bool']).columns
df[boolCols] = df[boolCols].astype(int)

print(nonNumeric)
print(boolCols)

for col in nonNumeric:
    if col != 'URL':
        df[col] = le.fit_transform(df[col])
        print(f'{col} encoded')



Ssl_Info dropped
Index(['TLD'], dtype='object')
Index(['Url_Shortening', 'Has_Suspicious_Params', 'Has_User_Info', 'Has_Ip',
       'Has_Port', 'Has_Fragment'],
      dtype='object')
TLD encoded


In [4]:
import joblib
joblib.dump(le, 'saved_models/label_encoder_24k.joblib')

['saved_models/label_encoder_24k.joblib']

In [5]:
df_nourl = df

correlation_with_label = df_nourl.corr(numeric_only=True)["label"].sort_values(ascending=False)

high_corr = correlation_with_label[abs(correlation_with_label) > 0.7]

feature_variance = df_nourl.drop(columns=["label"]).var()
unique_counts = df_nourl.drop(columns=["label"]).nunique()

high_corr, feature_variance.sort_values(), unique_counts.sort_values()

(label                1.000000
 Uses_HTTPS           0.951040
 entropy_subdomain    0.916160
 percent_Dots_Path   -0.821261
 Name: label, dtype: float64,
 percent_AtSymbol_Subdomain                      0.000000
 percent_Dash_Subdomain                          0.000000
 percent_Hex_Encoded_Characters_Subdomain        0.000000
 percent_Hash_Subdomain                          0.000000
 percent_Ampersand_Subdomain                     0.000000
                                                 ...     
 Length_Query                                 1136.220221
 URL_Length                                   2352.907935
 TLD                                         32618.755842
 Global_Ranking                                       NaN
 Domain_Age                                           NaN
 Length: 67, dtype: float64,
 Global_Ranking                      0
 Domain_Age                          0
 percent_Hash_Subdomain              1
 percent_Dash_Subdomain              1
 percent_TildeSymbol_Su

In [6]:
# Find columns with 0 variance
zero_variance = feature_variance[feature_variance == 0]
zero_variance



percent_Dash_Subdomain                      0.0
percent_AtSymbol_Subdomain                  0.0
percent_TildeSymbol_Subdomain               0.0
percent_Underscore_Subdomain                0.0
percent_Percent_Subdomain                   0.0
percent_Ampersand_Subdomain                 0.0
percent_Hash_Subdomain                      0.0
percent_Hex_Encoded_Characters_Subdomain    0.0
percent_Dash_Domain                         0.0
percent_AtSymbol_Domain                     0.0
percent_TildeSymbol_Domain                  0.0
percent_Underscore_Domain                   0.0
percent_Percent_Domain                      0.0
percent_Ampersand_Domain                    0.0
percent_Hash_Domain                         0.0
percent_Hex_Encoded_Characters_Domain       0.0
percent_Hash_Path                           0.0
percent_Hash_Query                          0.0
dtype: float64

In [7]:
df_nourl = df_nourl.drop(columns=zero_variance.index)


In [8]:
df_nourl

,label,URL_Length,Url_Shortening,entropy_subdomain,entropy_domain,Global_Ranking,Has_Suspicious_Params,Num_Languages,Uses_HTTPS,Has_User_Info,...,Num_Query_Params,percent_Dots_Query,percent_Dash_Query,percent_AtSymbol_Query,percent_TildeSymbol_Query,percent_Underscore_Query,percent_Percent_Query,percent_Ampersand_Query,percent_Hex_Encoded_Characters_Query,Has_Fragment
0,0,11,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
1,1,62,0,1.000000,3.926734,NaN,0,2,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
2,1,61,0,1.918296,1.921928,NaN,0,1,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
3,1,64,0,1.000000,3.819114,NaN,0,2,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
4,1,162,0,2.521641,-0.000000,NaN,0,1,0,0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,12,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
23996,1,39,0,2.251629,3.826875,NaN,0,1,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
23997,0,19,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
23998,0,21,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0


In [9]:
high_corr = correlation_with_label[abs(correlation_with_label) > 0.7]

high_corr.sort_values(ascending=False)

label                1.000000
Uses_HTTPS           0.951040
entropy_subdomain    0.916160
percent_Dots_Path   -0.821261
Name: label, dtype: float64

In [10]:
# df_nourl.drop(['Uses_HTTPS', 'Num_Dots_Path'], axis=1, inplace=True)

# df_nourl

unique_counts = df_nourl.nunique()



In [11]:
unique_counts.sort_values(ascending=False)


entropy_domain                          1398
TLD                                      588
entropy_subdomain                        453
percent_Digits_Path                      386
percent_Digits_Query                     327
URL_Length                               282
Length_Query                             201
percent_Hyphens_Path                     169
percent_Ampersand_Query                  168
percent_Dots_Query                       152
percent_Dots_Path                        147
percent_Underscore_Query                 131
percent_Dash_Query                       131
Length_Path                              123
percent_Hyphens_Query                    118
percent_Digits_Domain                     99
percent_Dash_Path                         75
percent_Underscore_Path                   75
percent_Percent_Query                     71
percent_Hex_Encoded_Characters_Query      71
percent_Hyphens_Domain                    56
percent_Digits_Subdomain                  49
Length_Dom

In [12]:
nan_counts = df_nourl.isna().sum()
nan_counts.sort_values(ascending=False)

df_nourl.drop(['Domain_Age'], axis=1, inplace=True)


In [13]:
df_nourl


,label,URL_Length,Url_Shortening,entropy_subdomain,entropy_domain,Global_Ranking,Has_Suspicious_Params,Num_Languages,Uses_HTTPS,Has_User_Info,...,Num_Query_Params,percent_Dots_Query,percent_Dash_Query,percent_AtSymbol_Query,percent_TildeSymbol_Query,percent_Underscore_Query,percent_Percent_Query,percent_Ampersand_Query,percent_Hex_Encoded_Characters_Query,Has_Fragment
0,0,11,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
1,1,62,0,1.000000,3.926734,NaN,0,2,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
2,1,61,0,1.918296,1.921928,NaN,0,1,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
3,1,64,0,1.000000,3.819114,NaN,0,2,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
4,1,162,0,2.521641,-0.000000,NaN,0,1,0,0,...,2,0.0,0.0,0.0,0.0,0.0,0.0,0.027778,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,12,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
23996,1,39,0,2.251629,3.826875,NaN,0,1,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
23997,0,19,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
23998,0,21,0,0.000000,0.000000,NaN,0,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0


In [14]:
low_variance = feature_variance[feature_variance < 0.01]
low_variance


Has_Suspicious_Params                       4.149478e-03
Has_User_Info                               3.748750e-04
percent_Digits_Subdomain                    5.943002e-04
percent_Hyphens_Subdomain                   1.454612e-05
percent_Dots_Subdomain                      1.572996e-05
percent_Dash_Subdomain                      0.000000e+00
percent_AtSymbol_Subdomain                  0.000000e+00
percent_TildeSymbol_Subdomain               0.000000e+00
percent_Underscore_Subdomain                0.000000e+00
percent_Percent_Subdomain                   0.000000e+00
percent_Ampersand_Subdomain                 0.000000e+00
percent_Hash_Subdomain                      0.000000e+00
percent_Hex_Encoded_Characters_Subdomain    0.000000e+00
Has_Ip                                      4.892697e-03
percent_Hyphens_Domain                      2.893923e-03
percent_Dots_Domain                         2.688687e-04
percent_Dash_Domain                         0.000000e+00
percent_AtSymbol_Domain        

In [15]:
for col in low_variance.index:
    if col in df_nourl.columns:
        df_nourl.drop(col, axis=1, inplace=True)

df_nourl



,label,URL_Length,Url_Shortening,entropy_subdomain,entropy_domain,Global_Ranking,Num_Languages,Uses_HTTPS,User_Info_Length,Num_Subdomains,Length_Subdomain,percent_Digits_Domain,Length_Domain,TLD,Path_Level,Length_Path,Length_Query,Num_Query_Params
0,0,11,0,0.000000,0.000000,NaN,1,0,0,0,0,0.000000,7,394,0,11,0,0
1,1,62,0,1.000000,3.926734,NaN,2,1,0,0,0,0.500000,2,149,1,11,0,0
2,1,61,0,1.918296,1.921928,NaN,1,1,0,0,0,0.000000,6,98,3,37,0,0
3,1,64,0,1.000000,3.819114,NaN,2,1,0,0,0,0.500000,2,149,1,13,0,0
4,1,162,0,2.521641,-0.000000,NaN,1,0,0,0,0,0.000000,7,98,4,103,36,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,12,0,0.000000,0.000000,NaN,1,0,0,0,0,0.000000,9,484,0,12,0,0
23996,1,39,0,2.251629,3.826875,NaN,1,1,0,0,0,0.000000,6,98,1,1,0,0
23997,0,19,0,0.000000,0.000000,NaN,1,0,0,0,0,0.000000,15,98,0,19,0,0
23998,0,21,0,0.000000,0.000000,NaN,1,0,0,0,0,0.000000,17,98,0,21,0,0


In [16]:
# df_nourl.to_csv('./newDatasets/clean_44k.csv', index=False)
low_var = df_nourl.loc[:, df_nourl.nunique() <= 2].columns
print("Low variance features:", low_var.tolist())
correlations = df_nourl.corr(numeric_only=True)["label"].sort_values(ascending=False)
print('Correlations:', correlations)

df_nourl.drop(['Global_Ranking','Path_Level'], axis=1, inplace=True)

Low variance features: ['label', 'Url_Shortening', 'Global_Ranking', 'Uses_HTTPS']
Correlations: label                    1.000000
Uses_HTTPS               0.951040
entropy_subdomain        0.916160
entropy_domain           0.663781
Path_Level               0.653069
URL_Length               0.416156
Url_Shortening           0.262276
Num_Query_Params         0.200454
Length_Query             0.136110
Num_Subdomains           0.125221
TLD                      0.093986
Length_Subdomain         0.074534
Length_Path              0.056465
percent_Digits_Domain    0.020321
User_Info_Length         0.017720
Num_Languages           -0.081116
Length_Domain           -0.249375
Global_Ranking                NaN
Name: label, dtype: float64


In [17]:
df_nourl.to_csv('./datasets/cleaned/clean_24k.csv', index=False)
df_nourl

,label,URL_Length,Url_Shortening,entropy_subdomain,entropy_domain,Num_Languages,Uses_HTTPS,User_Info_Length,Num_Subdomains,Length_Subdomain,percent_Digits_Domain,Length_Domain,TLD,Length_Path,Length_Query,Num_Query_Params
0,0,11,0,0.000000,0.000000,1,0,0,0,0,0.000000,7,394,11,0,0
1,1,62,0,1.000000,3.926734,2,1,0,0,0,0.500000,2,149,11,0,0
2,1,61,0,1.918296,1.921928,1,1,0,0,0,0.000000,6,98,37,0,0
3,1,64,0,1.000000,3.819114,2,1,0,0,0,0.500000,2,149,13,0,0
4,1,162,0,2.521641,-0.000000,1,0,0,0,0,0.000000,7,98,103,36,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,12,0,0.000000,0.000000,1,0,0,0,0,0.000000,9,484,12,0,0
23996,1,39,0,2.251629,3.826875,1,1,0,0,0,0.000000,6,98,1,0,0
23997,0,19,0,0.000000,0.000000,1,0,0,0,0,0.000000,15,98,19,0,0
23998,0,21,0,0.000000,0.000000,1,0,0,0,0,0.000000,17,98,21,0,0


In [18]:
removed_columns = set(oldColumns) - set(df_nourl.columns)
print("Columns removed from oldColumns to new columns:", removed_columns, '\n', 'Total columns removed:', len(removed_columns))

Columns removed from oldColumns to new columns: {'percent_Hash_Path', 'percent_Hash_Query', 'percent_Dots_Subdomain', 'percent_Hyphens_Domain', 'percent_Dash_Path', 'Has_Fragment', 'percent_AtSymbol_Domain', 'Ssl_Info', 'percent_TildeSymbol_Subdomain', 'Has_Ip', 'Domain_Age', 'percent_Ampersand_Query', 'percent_Hex_Encoded_Characters_Domain', 'percent_TildeSymbol_Domain', 'percent_AtSymbol_Path', 'percent_TildeSymbol_Path', 'percent_Percent_Query', 'percent_Ampersand_Path', 'percent_Underscore_Path', 'percent_AtSymbol_Subdomain', 'percent_Ampersand_Subdomain', 'percent_AtSymbol_Query', 'percent_DoubleSlash_Path', 'Has_User_Info', 'Has_Port', 'percent_Digits_Path', 'percent_Hex_Encoded_Characters_Path', 'percent_Digits_Query', 'percent_Dots_Domain', 'percent_Percent_Subdomain', 'Global_Ranking', 'percent_Hash_Subdomain', 'percent_Hex_Encoded_Characters_Query', 'percent_Underscore_Domain', 'percent_Percent_Path', 'percent_Hex_Encoded_Characters_Subdomain', 'percent_Hash_Domain', 'percent